In [7]:
from process_library import *
import os 

In [8]:
import warnings
warnings.filterwarnings('ignore') # Reduce number of messages/warnings displayed

In [9]:
print (f"Numpy version:{np.__version__}")
print (f"Pandas version:{pd.__version__}")


Numpy version:1.22.4
Pandas version:1.4.3


In [10]:
rng = np.random.RandomState(42)

# data loading 
path_to_parent_folder = "/".join( os.getcwd().split("/")[:-1] ) 
path_to_data = "data/pubchem_data_2023-12-05_343_samples.csv"
path = os.path.join(path_to_parent_folder,path_to_data)

df_all_data = pd.read_csv(path)
feature_names_involved_in_training = df_all_data.drop(['substituent','molecular_formula','Name','canonical_smiles','ΔVC-m','cid','ΔVC -p'],axis=1).columns

only_support_1d_arrays = ["Bayesian Ridge" ,"Huber Regressor", "Passive Aggressive Regressor","ARD Regression","Gradient Boosting","Support Vector Regression","Stochastic Gradient Descent"]

The following algorithms do not accept multiple columns targets : 
["Bayesian Ridge" ,"Huber Regressor", "Passive Aggressive Regressor","ARD Regression","Gradient Boosting","Support Vector Regression","Stochastic Gradient Descent"]

This is a constrain in the selection of the regressor algorithms

In [11]:
# There is three dirrefent targets 
results_dictionary = {}
for c in [0,1,2]:
    if c != 2 : 
        X_train, X_val, X_test, y_train, y_val, y_test = get_data_separated(df_all_data,random_state=rng,choice=c)
        # print (X_train.columns )
        X_train, X_val, X_test = scaling_data(X_train, X_val, X_test)
        target_feature_name = "-".join(y_train.columns.to_list()).replace(" ","")
        # print (target_feature_name)
        regression_models = load_regression_models()
        val_results_list = []
        for name, model in regression_models.items():
            # print (name)
            model.fit(X_train, y_train.to_numpy().ravel())
            val_results_list.append ( save_metrics_results(model=model,X_test=X_val,y_test=y_val,tag=name) ) 
        val_results_df = pd.concat(val_results_list)
        results_dictionary[target_feature_name]= val_results_df
    else :
        X_train, X_val, X_test, y_train, y_val, y_test = get_data_separated(df_all_data,random_state=rng,choice=c)
        # print (X_train.columns )
        X_train, X_val, X_test = scaling_data(X_train, X_val, X_test)
        target_feature_name = "-".join(y_train.columns.to_list()).replace(" ","")
        # print (target_feature_name)
        regression_models = load_regression_models() 
        val_results_list = []
        for name, model in regression_models.items():
            # print (name)
            if name not in only_support_1d_arrays:
                model.fit(X_train, y_train.to_numpy())
                val_results_list.append ( save_metrics_results(model=model,X_test=X_val,y_test=y_val,tag=name) )
        val_results_df = pd.concat(val_results_list)
        results_dictionary[target_feature_name]= val_results_df


In [12]:
for result in results_dictionary.keys():
    df_temp = results_dictionary[result].sort_values(by=["R^2", "MAE"],ascending=False)
    df_temp.to_csv(f"../data/performance_base_models_target_{result}.csv")